#**Knowledge Distillation Bi-LSTM Compressed Model - Class 19**


> In this notebook the Knowledge Distillation Bi-LSTM Compressed model has been done.






## **Importing Required Libraries**
*   Imporitng the required libraries and mounting the drive to train and evaluate the model

In [27]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report

## Bi-LSTM Model Compression
*   Importing the preprocessed and reshaped dataset from the local directory and Compressing the trained Bi-LSTM model.

In [37]:
# Preprocessed data path
# data_path = "<preprocessedrnn_data.npz file location for class 19>"
data_path = "/content/drive/MyDrive/AI_and_sustainability/AI_and_Sus_Final/Models_and_Result/Class_19/Preprocessed data/preprocessedrnn_data19.npz"
3
# Model saved path
# model_path = "<saved LSTM_19.h5 model file location for class 19>"
model_path = "/content/drive/MyDrive/AI_and_sustainability/AI_and_Sus_Final/Models_and_Result/Class_19/Result/LSTM_19.h5"

# Save Path
# save_path = "<Path to save the model results>"
save_path = "/content/drive/MyDrive/AI_and_sustainability/AI_and_Sus_Final/Models_and_Result/Class_19/Result_Compressed"

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Load teacher model
teacher_model = tf.keras.models.load_model(model_path)

# Load training data
data = np.load(data_path)
X_train = data['X_train1']
y_train = data['y_train']  # One-hot encoded

# Reshape for LSTM input if needed
if len(X_train.shape) == 2:
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

# Generate soft labels (probabilities) from teacher
teacher_soft_labels = teacher_soft_labels = teacher_model.predict(X_train, batch_size=32)




179021/179021 ━━━━━━━━━━━━━━━━━━━━ 403s 2ms/step


## Compressed Model Training


In [6]:
# Define student model
def create_student_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=False, input_shape=input_shape),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Instantiate student model
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = y_train.shape[1]
student_model = create_student_model(input_shape, num_classes)

# Define distillation wrapper
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.student = student
        self.teacher = teacher

    def compile(self, optimizer, metrics, temperature=3.0, alpha=0.5):
        super().compile()
        self.optimizer = optimizer
        self.metrics_list = metrics
        self.temperature = temperature
        self.alpha = alpha

    def train_step(self, data):
        x, y_true = data
        teacher_soft = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            y_pred = self.student(x, training=True)
            y_pred_soft = tf.nn.softmax(y_pred / self.temperature)
            teacher_soft_temp = tf.nn.softmax(teacher_soft / self.temperature)

            distill_loss = tf.keras.losses.KLDivergence()(teacher_soft_temp, y_pred_soft)
            hard_loss = tf.keras.losses.CategoricalCrossentropy()(y_true, y_pred)
            loss = self.alpha * hard_loss + (1 - self.alpha) * distill_loss

        grads = tape.gradient(loss, self.student.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        for metric in self.metrics_list:
            metric.update_state(y_true, y_pred)

        return {m.name: m.result() for m in self.metrics_list}

    def test_step(self, data):
        x, y_true = data
        y_pred = self.student(x, training=False)
        for metric in self.metrics_list:
            metric.update_state(y_true, y_pred)
        return {m.name: m.result() for m in self.metrics_list}

# Instantiate distiller and compile
distiller = Distiller(student_model, teacher_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    temperature=3.0,
    alpha=0.5
)

# Train student with knowledge distillation
distiller.fit(X_train, y_train, epochs=10, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 830s 5ms/step - categorical_accuracy: 0.8163
Epoch 2/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 892s 5ms/step - categorical_accuracy: 0.9588
Epoch 3/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 941s 5ms/step - categorical_accuracy: 0.9911
Epoch 4/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 836s 5ms/step - categorical_accuracy: 0.9977
Epoch 5/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 818s 5ms/step - categorical_accuracy: 0.9978
Epoch 6/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 870s 5ms/step - categorical_accuracy: 0.9979
Epoch 7/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 841s 4ms/step - categorical_accuracy: 0.9980
Epoch 8/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 889s 5ms/step - categorical_accuracy: 0.9980
Epoch 9/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 829s 4ms/step - categorical_accuracy: 0.9981
Epoch 10/10
179021/179021 ━━━━━━━━━━━━━━━━━━━━ 776s 4ms/step - categorical_accuracy: 0.9981


## Evaluating and Saving the Model



In [31]:
import time  # Import the time module

def evaluate_and_save_model(model, X_test, y_test_categorical, label_classes, save_path, model_name):

    # Ensure directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Record the start time
    start_time = time.time()

    # Evaluate the model on test data
    loss, accuracy = model.evaluate(X_test, y_test_categorical, verbose=0)

    # Predict class probabilities
    y_pred_probs = model.predict(X_test, verbose=0)

    # Get predicted class indices and true class indices
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    y_true_classes = np.argmax(y_test_categorical, axis=1)

    # Compute evaluation metrics
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    report = classification_report(y_true_classes, y_pred_classes, target_names=[str(c) for c in label_classes])
    conf_matrix = confusion_matrix(y_true_classes, y_true_classes)

    # Record the end time
    end_time = time.time()

    # Calculate the evaluation time
    eval_time = end_time - start_time

    # Print metrics
    print(f"\nModel Evaluation:")
    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (weighted): {precision:.4f}")
    print(f"Recall (weighted): {recall:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Save results dictionary as JSON
    results = {
        "model_name": model_name,
        "accuracy": float(accuracy),
        "loss": float(loss),
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "evaluation_time_sec": float(eval_time),  # Now eval_time is calculated
        "confusion_matrix": conf_matrix.tolist(),
        "true_labels": y_true_classes.tolist(),
        "predicted_labels": y_pred_classes.tolist(),
        "predicted_probabilities": y_pred_probs.tolist(),
    }

    with open(save_path, "w") as f:
        json.dump(results, f, indent=4)

    print(f"\nResults saved to: {save_path}")


In [32]:
model_name = "Distilled Bi-LSTM"

evaluate_and_save_model(
    model=student_model,
    X_test=data['X_test1'],
    y_test_categorical=data['y_test'],
    label_classes=data['label_classes'],
    save_path=save_path,
    model_name=model_name  # Pass model_name here
)



Model Evaluation:
Loss: 1.1550
Accuracy: 0.9578
Precision (weighted): 0.9306
Recall (weighted): 0.9578
F1 Score (weighted): 0.9424
Classification Report:
                         precision    recall  f1-score   support

                 Benign       0.98      0.96      0.97     37607
              DDoS-ICMP       1.00      1.00      1.00    349699
               DDoS-SYN       0.98      0.99      0.99    172397
               DDoS-TCP       0.99      1.00      1.00    182598
               DDoS-UDP       1.00      1.00      1.00    362070
               DoS-ICMP       1.00      1.00      1.00     98432
                DoS-SYN       0.96      0.99      0.98     98595
                DoS-TCP       1.00      1.00      1.00     82096
                DoS-UDP       0.74      1.00      0.85    137553
MQTT-DDoS-Connect_Flood       0.00      0.00      0.00     41916
MQTT-DDoS-Publish_Flood       0.00      0.00      0.00      8416
 MQTT-DoS-Connect_Flood       1.00      1.00      1.00      3131

In [38]:
# Create directory
os.makedirs(save_path, exist_ok=True)

# Define filename
model_filename = 'LSTM_Distilled.h5'
full_save_path = os.path.join(save_path, model_filename)

# Save the Keras model properly
student_model.save(full_save_path)
print(f"Model saved successfully to: {full_save_path}")


Model saved successfully to: /content/drive/MyDrive/AI_and_sustainability/AI_and_Sus_Final/Models_and_Result/Class_19/Result_Compressed/LSTM_Distilled.h5
